This notebook explores a new source of information we need to work through for the "Assessment Analytical Framework" we are developing to support mineral resource assessments in the USGS. The framework consists of various technologies designed to produce analysis-ready data for the assessment process. A big part of that is the development of workflows, as automated as possible, that pull source data and information and "mobilize" them up for use.

For many years, we've been collecting and working with a type of technical report required of mining companies based in Canada (NI 43-101 Technical Report) as a source for mineral deposit type information and other geoscientific details. The Securities and Exchange Commission (SEC) in the U.S. now also [requires this type of information](https://www.sec.gov/corpfin/secg-modernization-property-disclosures-mining-registrants) to be filed as a regular submission from any publicly traded mining companies in the U.S. Similar to the NI 43-101 Reports that we have assembled for use in a Zotero collection, we also need to start pulling and organizing the  S-K 1300 Technical Reports required by the SEC. All of the same basic requirements apply:

* Provide an online locale for the reports that is easily accessible and manageable by assessment geologists and science support staff
* Provide for consistent, publicly citable references in reports and articles, including ensuring long-term viability of the citable references
* Provide a mechanism for routine annotation of reports by assessment geologists and science support staff
* Provide a means for the report contents to be fed into AI/ML pipelines for further processing to identify/link entities (mineral deposit sites, etc.) and automate aspects of turning information into analyzable data

As an improvement over the situation with the [SEDAR](https://www.sedar.com/) source for NI 43-101 Reports where we have a web site expressly protected against robot interfaces, the SEC's EDGAR platform provides some decent [direct data interfaces](https://www.sec.gov/edgar/sec-api-documentation) to support automated retrieval of the company submissions we are interested in. Data from the SEC is of great interest in a lot of different circumstances, and their data handling and availability seems to be subsequently quite mature.

Our initial starting point is likely the set of two bulk data downloads compiled nightly. Both of these are zip files with JSON documents for each unique identified company over which the SEC exercises regulatory authority. One of these contains a set of "company facts," and the other lists the submissions the companies have filed with the SEC (which should include the S-K 1300 Technical Reports).

Many times, bulk downloads are a much better starting point or even long-term source for data from an origin point like this. They give us a big tranche of data that we have to download initially and spin up into some kind of our own data infrastructure for use, but they help us avoid a number of problems that can arise when trying to operate against what are often purpose-built APIs that may be less than fully stable or may not provide the most efficient means for us to obtain the answers we want. In this case as with many others, we really only want a small slice of what the overall source contains (the SEC regulates many companies, only some of which are involved in mineral resources).

The nighly files appear to contain current information, and it's not yet clear if we will need to tap some other source (like the live APIs) for historic facts (if we are interested in any of those) or submissions. One way or the other, we need to narrow in on those companies required to file S-K 1300 Technical Reports. The remainder of this notebook starts to crack open the bulk downloads to explore how we might go about effectively working this source into our analytical framework.

In [ ]:
!pip install nested_lookup requests-random-user-agent

In [1]:
import json
from glob import glob
from nested_lookup import nested_lookup
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Company Facts
The company facts download pulled on 8/15/2022 contains 16,410 files, presumably representing each of the uniquely identified companies with a "CIK" identifier that the SEC tracks. I pulled just a few of these to take a look.

In [2]:
sample_companyfacts_files = glob('/home/skybristol/experiments/data/sec_edgar/companyfacts/*')
sample_companyfacts_files

['/home/skybristol/experiments/data/sec_edgar/companyfacts/CIK0001892274.json',
 '/home/skybristol/experiments/data/sec_edgar/companyfacts/CIK0001001838.json',
 '/home/skybristol/experiments/data/sec_edgar/companyfacts/CIK0001898496.json',
 '/home/skybristol/experiments/data/sec_edgar/companyfacts/CIK0001885827.json']

In [3]:
sample_companyfacts = json.load(open(sample_companyfacts_files[0], "r"))

Beyond the CIK identifier the SEC uses, there is a name for a company, which is likely a whole other issue we may or may not need to deal with. There might be some scenarios for downstream uses of these data where we need to effectively link together multiple sources of data tied to a given company entity. There might be different derivations of the names to deal with or temporal issues where company names/identifiers change.

In [4]:
sample_companyfacts["entityName"]

'Visionary Education Technology Holdings Group Inc.'

## Company Facts leading to "Mining Registrant"
The bulk of the data in the company facts information is contained in a part of the structure coded as "facts/us-gaap." A short bit of digging shows that the properties here track to a [standard financial reporting taxonomy](https://www.fasb.org/xbrl) the SEC has [adopted](https://www.sec.gov/info/edgar/edgartaxonomies.shtml#USGAAP2019). This gives us a nice starting point to understand the data and figure out what we can use for what purposes.

We can approach our problem from a couple different perspectives. It might be useful to identify all of the companies who are considered "mining registrants." This is the term the SEC uses in its [guidance](https://www.sec.gov/corpfin/secg-modernization-property-disclosures-mining-registrants). So, one question is, how do we figure out which companies (of the, presumably, 16,410) are mining registrants? We need to do some digging to see if there is something specific or a combination of attributes that would help us work this out.

In [5]:
list(sample_companyfacts["facts"]['us-gaap'].keys())

['AccountsPayableCurrent',
 'AccountsReceivableGross',
 'AccountsReceivableNet',
 'AccountsReceivableNetCurrent',
 'AccountsReceivableRelatedParties',
 'AccountsReceivableRelatedPartiesCurrent',
 'AccruedIncomeTaxes',
 'AccruedIncomeTaxesCurrent',
 'AccruedLiabilitiesCurrent',
 'AccruedLiabilitiesForCommissionsExpenseAndTaxes',
 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
 'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'AdditionalPaidInCapital',
 'AllowanceForDoubtfulAccountsReceivable',
 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount',
 'AssetImpairmentCharges',
 'Assets',
 'AssetsCurrent',
 'BusinessCombinationPriceOfAcquisitionExpected',
 'BusinessCombinationRecognizedIdentifiableAssetsAcquiredAndLiabilitiesAssumedIndefiniteLivedIntangibleAssets',
 'BusinessCombinationRecognizedIdentifiableAssetsAcquiredGoodwillAndLiabilitiesAssumedNet',
 'CapitalExpenditureDiscontinuedOperations',
 'Cash',
 'CashCashEquivalentsRestrictedC

In [6]:
sample_companyfacts["facts"]['us-gaap']['Assets']

{'label': 'Assets',
 'description': 'Sum of the carrying amounts as of the balance sheet date of all assets that are recognized. Assets are probable future economic benefits obtained or controlled by an entity as a result of past transactions or events.',
 'units': {'USD': [{'end': '2021-03-31',
    'val': 13667102,
    'accn': '0001683168-22-005640',
    'fy': 2022,
    'fp': 'FY',
    'form': '20-F',
    'filed': '2022-08-12',
    'frame': 'CY2021Q1I'},
   {'end': '2022-03-31',
    'val': 36226584,
    'accn': '0001683168-22-005640',
    'fy': 2022,
    'fp': 'FY',
    'form': '20-F',
    'filed': '2022-08-12',
    'frame': 'CY2022Q1I'}]}}

In [7]:
sample_companyfacts["facts"]['us-gaap']['Land']

{'label': 'Land',
 'description': 'Amount before accumulated depletion of real estate held for productive use, excluding land held for sale.',
 'units': {'USD': [{'end': '2022-03-31',
    'val': 4400000,
    'accn': '0001683168-22-005640',
    'fy': 2022,
    'fp': 'FY',
    'form': '20-F',
    'filed': '2022-08-12',
    'frame': 'CY2022Q1I'}]}}

# Submissions
The main thing we are after from the SEC EDGAR source are the S-K 1300 Technical Reports. These should be contained and referenced in the submissions part of the EDGAR data model. Here, we grab up a few of these files to take a look at what they contain.

In [8]:
sample_submissions_files = glob('/home/skybristol/experiments/data/sec_edgar/submissions/*')
sample_submissions_files

['/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001942630.json',
 '/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001001838.json',
 '/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001942661.json',
 '/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001001838-submissions-001.json',
 '/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001942693.json']

In [9]:
sample_submissions = json.load(open(sample_submissions_files[0], "r"))

In [10]:
sample_submissions

{'cik': '1942630',
 'entityType': 'other',
 'sic': '',
 'sicDescription': '',
 'insiderTransactionForOwnerExists': 0,
 'insiderTransactionForIssuerExists': 0,
 'name': 'MOV Fund III, a series of DV Deals, LP',
 'tickers': [],
 'exchanges': [],
 'ein': '000000000',
 'description': '',
 'website': '',
 'investorWebsite': '',
 'category': '',
 'fiscalYearEnd': '1231',
 'stateOfIncorporation': 'DE',
 'stateOfIncorporationDescription': 'DE',
 'addresses': {'mailing': {'street1': '119 SOUTH MAIN STREET',
   'street2': 'SUITE 220',
   'city': 'SEATTLE',
   'stateOrCountry': 'WA',
   'zipCode': '98104',
   'stateOrCountryDescription': 'WA'},
  'business': {'street1': '119 SOUTH MAIN STREET',
   'street2': 'SUITE 220',
   'city': 'SEATTLE',
   'stateOrCountry': 'WA',
   'zipCode': '98104',
   'stateOrCountryDescription': 'WA'}},
 'phone': '3603409337',
 'flags': '',
 'formerNames': [],
 'filings': {'recent': {'accessionNumber': ['0001942630-22-000002'],
   'filingDate': ['2022-08-12'],
   'repo

There are actually some interesting additional useful company details contained in this structure that may or may not be in the company facts information. I'm particularly intrigued by a potential array of formerNames that could be quite helpful.

We're going to need to dig through some more examples to figure out how to use the submission information effectively. It looks like we might be able to find what we need within the filings part of the document structure if we can suss out what an S-K 1300 filing looks like. We can do this by digging up a company that we know has to file the reports, tracing them through the SEC EDGAR data structures, and seeing if we can learn the query pattern we could use. We'll start that next using an [example](https://www.sec.gov/Archives/edgar/data/1001838/000155837022002995/scco-20211231ex968d77397.pdf) Mike Zientek provided.

# Mining Registrant

I tracked down the CIK identifier for the [Southern Copper Corporation](https://www.sec.gov/edgar/browse/?CIK=1001838&owner=exclude) that was the company behind an example we have. I added these into my sample data here, and we'll take a look.

Note: This code block contains simple examples for how we might progress through core logic to run part of our data management operation. First, the SEC EDGAR system is like a lot of data systems where identifiers can take on different logical forms depending on where they are being referenced. In using the web interface for an EDGAR search, we see a reference to a CIK identifier in the URL like this - CIK=1001838. That's pretty much a numeric ID. In the bulk download, we have JSON files that have a "zero-padded" 10-character string form of the CIK ID preceded by "CIK." That's simple logic to deal with in code, but we might want to formalize it somewhere so that we can quickly transform into different forms of identifiers specific to a given source. We can start by building out functions for this and other things we need to do and then eventually move to packaging those functions into a deployable library of some kind. I provide a couple other stubs for functional code to start working through EDGAR source data.

In [11]:
def cik_str(cik_id, prefix="CIK"):
    """
    Simple function to transform an SEC EDGAR CIK identifier into a string.
    """
    padded_id = str(cik_id).zfill(10)
    return prefix + padded_id

def cik_data_files(
    cik_id, 
    source_type="companyfacts", 
    source_path="/home/skybristol/experiments/data/sec_edgar"
):
    cik_id_str = cik_str(cik_id)
    data_files = glob("/".join([source_path, source_type, cik_id_str + "*"]))
    return data_files

def load_data_file(path=None, cik_id=None, source_type=None):
    if path is not None:
        return json.load(open(path, "r"))
    
    if cik_id is not None:
        return [json.load(open(p, "r")) for p in cik_data_files(cik_id, source_type)]
            

In a future scenario, we might be able to develop something like a "mining company registry" as a data component in our analytical framework. That might include U.S. mining companies as well as Canadian and other jurisdictions and could have various kinds of information that we need to use, including things like important identifiers for companies that would facilitate processes for going after up to date information. Here, we just put down one of those we know about so we can use our functions to go after source data for the Southern Copper Corp.

In [12]:
sample_cik_id = 1001838

Looking at the file structure from the bulk data download, we see that we have a single source JSON file for company facts, but in this case, we have two different submissions files. You can see this -submissions-00{n}.json dynamic for other companies as well.

In [13]:
sample_company_facts_source_files = cik_data_files(sample_cik_id)
sample_company_submission_source_files = cik_data_files(sample_cik_id, "submissions")

display(sample_company_facts_source_files)
display(sample_company_submission_source_files)

['/home/skybristol/experiments/data/sec_edgar/companyfacts/CIK0001001838.json']

['/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001001838.json',
 '/home/skybristol/experiments/data/sec_edgar/submissions/CIK0001001838-submissions-001.json']

In [14]:
sample_company_facts = load_data_file(sample_company_facts_source_files[0])

After fiddling around looking through company facts for our sample case, I don't know what all utility might come from these records. There are a couple of interesting properties in the data like RevenueMineralSales and ExplorationExpenseMining. These show up in the company facts file for our sample company. The facts reported like this are tied to a beginning and end date and seem to contain a tally of what a given company is needing to report to the SEC. Reporting mineral sales revenue and mining-related expenses might be useful for some things we might imagine. However, the labels for these indicate the properties were deprecated in 2018, so maybe they won't be reliable as a way to identify mining companies, at least for later filings.

I went poking around the "Financial Accounting Standards Board," which is where the references point for the schema used in the SEC EDGAR source data. As expected, I don't see this property listed in the current 2022 standard that seems to be in review status, but I also don't see if in the listing of deprecated properties.

Side note: The FASB is a non-profit indistry organization that nevertheless charges a fee for full access to the standards. I found a review version of the latest 2022 standard to poke around, but all of it's URI references (unique, resolvable identifiers for individual properties) loop through a paywall. So, it's not really an open standard or accessible reference. I get the fact that a non-profit has to fund its most important operations somehow, but charging for basic access to the definitions of terms in a standard like this is pretty heinous. Government agencies like the SEC need deeper level policy and rules that push them to make anything behind what they are adopting as a standard fully open and accessible as well as the data themselves.

Looking into the submissions information, there are actually more useful descriptive properties about the company in the "header" part of the data structure. Most notably, I found sic and sicDescription in our example case and tracked those down as Standard Industry Code from the SEC (something our NMIC colleagues probably already know all about).

In [15]:
sample_company_submissions = load_data_file(
    cik_id=sample_cik_id,
    source_type="submissions"
)

In [16]:
nested_lookup("sic", sample_company_submissions, wild=True)

['1000', 'Metal Mining']

Here's a quick look at the SIC values in the [table from the SEC](https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list)with "MINING" in the industry title. Some of these probably don't apply, and there are probably some others without mining in the title we should consider. Someone who knows this field should take a look at this and see if we can come up with the SIC codes for companies who will be the "mining registrants" that we care about. We can also simply bypass that idea and go straight to looking for S-K 1300 report filings, which I'll get into next, but I think it would still be useful to understand the classification and come up with a dynamic registry of mining companies from the SEC's perspective that we want to know something about in our framework.

In [17]:
sec_sic_tables = pd.read_html("https://www.sec.gov/corpfin/division-of-corporation-finance-standard-industrial-classification-sic-code-list")

sec_sic_tables[0].loc[sec_sic_tables[0]["Industry Title"].str.contains("MINING")]

,SIC Code,Office,Industry Title
5,1000,Office of Energy & Transportation,METAL MINING
8,1220,Office of Energy & Transportation,BITUMINOUS COAL & LIGNITE MINING
9,1221,Office of Energy & Transportation,BITUMINOUS COAL & LIGNITE SURFACE MINING
14,1400,Office of Energy & Transportation,MINING & QUARRYING OF NONMETALLIC MINERALS (NO...
152,3530,Office of Technology,"CONSTRUCTION, MINING & MATERIALS HANDLING MACH..."
154,3532,Office of Technology,MINING MACHINERY & EQUIP (NO OIL & GAS FIELD M...
293,5082,Office of Trade & Services,WHOLESALE-CONSTRUCTION & MINING (NO PETRO) MAC...


Looking at the submissions bulk source data, the stuff that comes from the "secondary" file appears to be a continuation of older filings without the company information in the "primary" file. This must have something to do with how the EDGAR system is pushing the data out.

The data on files are organized into arrays of values within the keys that are presumably ordered within the JSON output so that we can load them into a different structure like a table. At least each property has the same number of items in its array.

In [18]:
for k,v in sample_company_submissions[0]["filings"]["recent"].items():
    print(k, len(v))

print()
    
for k,v in sample_company_submissions[1].items():
    print(k, len(v))

accessionNumber 1001
filingDate 1001
reportDate 1001
acceptanceDateTime 1001
act 1001
form 1001
fileNumber 1001
filmNumber 1001
items 1001
size 1001
isXBRL 1001
isInlineXBRL 1001
primaryDocument 1001
primaryDocDescription 1001

accessionNumber 63
filingDate 63
reportDate 63
acceptanceDateTime 63
act 63
form 63
fileNumber 63
filmNumber 63
items 63
size 63
isXBRL 63
isInlineXBRL 63
primaryDocument 63
primaryDocDescription 63


Putting things together into something more familiar like a table structure, we can use Pandas and concat the two primary data structures.

In [19]:
sample_company_filings = pd.concat([
    pd.DataFrame(sample_company_submissions[0]["filings"]["recent"]),
    pd.DataFrame(sample_company_submissions[1])
])

In [20]:
sample_company_filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001415889-22-008188,2022-08-03,2022-08-03,2022-08-03T16:16:09.000Z,,4,,,,4942,0,0,xslF345X03/form4-08032022_040847.xml,
1,0001415889-22-008184,2022-08-03,2022-08-03,2022-08-03T16:16:07.000Z,,4,,,,4836,0,0,xslF345X03/form4-08032022_040818.xml,
2,0001415889-22-008187,2022-08-03,2022-08-03,2022-08-03T16:16:03.000Z,,4,,,,4899,0,0,xslF345X03/form4-08032022_040841.xml,
3,0001415889-22-008183,2022-08-03,2022-08-03,2022-08-03T16:15:56.000Z,,4,,,,4825,0,0,xslF345X03/form4-08032022_040812.xml,
4,0001415889-22-008186,2022-08-03,2022-08-03,2022-08-03T16:15:51.000Z,,4,,,,4823,0,0,xslF345X03/form4-08032022_040830.xml,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0000007649-96-000014,1996-11-14,1996-09-30,1996-11-14T00:00:00.000Z,,10-Q,001-14066,96664242,,65904,0,0,,
59,0000950127-96-000096,1996-10-08,,1996-10-08T00:00:00.000Z,,4,001-14066,96640814,,10289,0,0,,
60,0000007649-96-000010,1996-08-14,1996-06-30,1996-08-14T00:00:00.000Z,,10-Q,001-14066,96614000,,63202,0,0,,
61,0000950103-96-000875,1996-05-14,1996-03-31,1996-05-14T00:00:00.000Z,,10-Q,001-14066,96564362,,45084,0,0,,


Our basic objective at this point is, give me something that points toward a filing for an "S-K 1300 Technical Report" from this company, and tell me how to get it. There are a couple properties in the filings data here that seem like they could give us that clue, but a look at the unique values doesn't seem to turn anything up.

Looking at the EDGAR user interface for our [sample company](https://www.sec.gov/edgar/browse/?CIK=1001838&owner=exclude) that should have one of these reports, I do notice an interesting thing that it shows and links to what appears to be the exact same JSON file we pulled down via the bulk download, which tells me we're probably connecting dots appropriately. Initially, I tried to look at the "form" and "primaryDocDescription" properties to see if I could suss out what classifies the files we're interested in. There aren't any values there that make sense for that, though.

Looking at the link that Mike Zientek sent along earlier, though, there's a pattern in the URL that looks a little like the accessionNumber from the EDGAR data, which seems reasonable. 

https://www.sec.gov/Archives/edgar/data/1001838/000155837022002995/scco-20211231ex968d77397.pdf

Parsing the value to match that seems to include some numbers, a two character year, and some more numbers, we do turn up a record from the URL pattern. From this point of view, this seems to show up as a "10-K (Annual Report)" in the terminology used in the EDGAR system. Digging around in the user interface does land us on what appears to be a robust [archival accession](https://www.sec.gov/ix?doc=/Archives/edgar/data/0001001838/000155837022002995/scco-20211231x10ka.htm) for this whole report and its exhibits, which is where I think we'll get to the technical reports when those exist.

Somehow, we need to programmaticaly identify that a company has a technical report includes with a required filing and suss out the link to the actual PDF file. With that, we could create reasonable bibliographic metadata for the citation, include the link to the original source, and stash our own copy of the PDF if we wanted to. However, I'm not yet seeing a clear pathway to get there from what the SEC EDGAR system puts out in their API and bulk data downloads.

In [21]:
sample_company_filings[sample_company_filings.accessionNumber == "0001558370-22-002995"]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
29,0001558370-22-002995,2022-03-08,2021-12-31,2022-03-07T21:13:56.000Z,34,10-K/A,001-14066,22720169,,235002213,1,1,scco-20211231x10ka.htm,10-K/A


In [22]:
sample_company_filings[sample_company_filings.form.str.contains("10-K")]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
28,0001558370-22-002999,2022-03-08,2021-12-31,2022-03-07T21:32:57.000Z,34,10-K/A,001-14066,22720189,,56297247,1,1,scco-20211231x10ka.htm,10-K/A
29,0001558370-22-002995,2022-03-08,2021-12-31,2022-03-07T21:13:56.000Z,34,10-K/A,001-14066,22720169,,235002213,1,1,scco-20211231x10ka.htm,10-K/A
38,0001558370-22-002968,2022-03-07,2021-12-31,2022-03-07T17:15:56.000Z,34,10-K,001-14066,22719008,,28183471,1,1,scco-20211231x10k.htm,10-K
39,0001104659-22-028871,2022-03-02,2021-12-31,2022-03-01T19:58:00.000Z,34,NT 10-K,001-14066,22700710,,32933,0,0,tm228097d1_nt10q.htm,NT 10-K
89,0001558370-21-001894,2021-02-26,2020-12-31,2021-02-25T20:39:44.000Z,34,10-K,001-14066,21683099,,26898648,1,1,scco-20201231x10k.htm,10-K
150,0001104659-20-053164,2020-04-29,2019-12-31,2020-04-29T13:37:18.000Z,34,10-K/A,001-14066,20829082,,582789,1,1,tm2016906-1_10ka.htm,FORM 10-K/A
153,0001558370-20-001781,2020-02-28,2019-12-31,2020-02-28T16:24:56.000Z,34,10-K,001-14066,20670617,,26887010,1,1,scco-20191231x10k32f1e0.htm,10-K
182,0001047469-19-000845,2019-03-01,2018-12-31,2019-03-01T16:48:11.000Z,34,10-K,001-14066,19650189,,25082649,1,0,a2237832z10-k.htm,10-K
211,0001047469-18-001211,2018-03-01,2017-12-31,2018-03-01T16:36:47.000Z,34,10-K,001-14066,18657993,,23513496,1,0,a2234122z10-k.htm,10-K
258,0001047469-17-001143,2017-03-01,2016-12-31,2017-03-01T17:22:54.000Z,34,10-K,001-14066,17655741,,23790039,1,0,a2231031z10-k.htm,10-K


It appears that the API and the bulk data may not necessarily give us a pathway to pull the actual technical report files themselves. We can get to the level of identifying a mining company and finding the annual filing where S-K 1300 Technical Reports might be included as one of many "exhibits." We'd have to then find a way to crack open the filing and 

In [23]:
s = requests.Session()
s.headers['User-Agent'] = "U.S. Geological Survey;https://www.usgs.gov"

r = s.get("https://www.sec.gov/Archives/edgar/data/0001001838/000155837022002995/scco-20211231x10ka.htm")

sample_soup = BeautifulSoup(r.content, 'html.parser')

tables = sample_soup.findAll("table")

In [24]:
for tr in tables[7].findAll("tr"):
    for td in tr.findAll("td"):
        print(td.text)

23.8
​
Consent of Qualified Persons for Technical Report Summary of Mineral Reserves and Mineral Resources for the La Caridad Mine (Filed as Exhibit 23.8 to the Company’s Annual Report on Form 10-K for the fiscal year ended December 31, 2021 and incorporated herein by reference).
​
​
23.9
​
Consent of Qualified Persons for Technical Report Summary of Mineral Resources for the Pilares Project (Filed as Exhibit 23.9 to the Company’s Annual Report on Form 10-K for the fiscal year ended December 31, 2021 and incorporated herein by reference).
​
​
23.10
​
Consent of Qualified Persons for Technical Report Summary of Mineral Reserves and Mineral Resources for the El Pilar Project (Filed as Exhibit 23.10 to the Company’s Annual Report on Form 10-K for the fiscal year ended December 31, 2021 and incorporated herein by reference).
​
​
23.11
​
Consent of Qualified Persons for Technical Report Summary of Mineral Reserves and Mineral Resources for the El Arco Project (Filed as Exhibit 23.11 to the 

# Next

With a little further digging, I figured out that the API or bulk data is only going to get us so far toward our objective of finding and keeping up with every new S-K 1300 Technical Report filed with the SEC by US mining companies. We are going to need to identify the actual company CIK identifiers for mining companies that we want to follow up on as that will make everything else flow more efficiently. We can do that by nailing down the SIC codes that apply and working through the bulk data download periodically to find any new companies.

The bulk data, which we can retrieve individually for all known CIK identifiers in our registry, for submissions will give us accession numbers and other details for any 10-K annual reports, which is where we'll find our technical reports as part of the filings. The data also point us toward an HTML landing page that lists out the contents included in the annual report filing. Unfortunately, it looks like we have to retrieve those individually and run them through a web scraper to pull them in as data, but it should give us the basic table of contents and useful titles for the technical reports, which are all listed with "96" codes as seen in the above example. I had to fiddle with how the request was being made as the SEC web site protects against robots that don't declare themselves in a way that passes their robot gatekeeper.

Another hurdle I haven't quite figured out yet is how to deal with cases where the actual URL link to a filed PDF as part of an accession is not included in the accession's index page. That's the case with the example above. There are actual files linked to from a different dynamic page for the accession and its amendments. Here's an example:

https://www.sec.gov/Archives/edgar/data/1001838/000155837022002995/0001558370-22-002995-index.htm

These URLs are also patterned on the CIK identifier and the accession identifier, but I have to work out just how consistent these are. Those listings seem to include the "type" which we can tie to the index with the titles and the link to the actual PDF, which we can pull as a download.

So, fundamentally, the SEC EDGAR API and their bulk downloads get us much of the way there to writing an automated workflow. There are some brittle parts that we'll have to keep an eye on. The PDFs also do not appear to be any better than the ones we get from the Canadian system. They are not professionally produced and do not contain good embedded metadata. We will at least get good titles and dates from the API route we are taking. It's also possible we could do something creative on authorship with the separate documents detailing cognizant experts, but that looks sketchy.

Once the remaining details on this are sussed out, I can see a clear pathway to finish out a process of dynamically contributing to a new Zotero group library for these documents. Since we have to handle these first through a cloud-based pipeline, we might end up dropping PDFs to our own storage location on these first and then upload to Zotero. I still think that any cases like SEDAR where we have a human needing to do the work, we should just drop them to Zotero first as the primary interface.